In [ ]:
# 파일이름 : model_prediction.ipynb
# 코드설명 : MLOps에 탑재하기 위한 최종 선정 모델의 물성 및 불량여부 예측 예제 코드
# 입/출력 : 물성별(물성값, 불량여부) 데이터 세트와 예측 대상별 학습된 모델들 / 테스트 데이터에 대한 물성(들)의 예측 결과
# 유의 사항 : 선정된 모델의 기술명칭 및 대상 물성 명칭을 설정 필요
# 최종수정 : 2023년 11월 20일
# 제 작 자 : 홍민성 (mshong@micube.co.kr), 맹영준 (myj6223@micube.co.kr)
# Copyright : MICUBE Solution, Inc.

# python 버전 : 3.9.18
# autogluon 버전 : 0.8.2

In [1]:
# 관련 라이브러리 로드
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", 500)
pd.set_option('display.max_rows', 50)

import os
import random
import warnings
warnings.filterwarnings(action='ignore')

from sklearn.model_selection import train_test_split

# autoML
from autogluon.tabular import TabularPredictor

# 동일 결과 재생성을 위한 랜덤성 고정
seed = 42
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)

## 예측 결과 보기
- 예측 대상별로 최적 모델들을 분석, 최적의 모델 3-4개 선정

In [12]:
%%time
loop_ls = [('regression','LightGBM_BAG_L2'),('binary','XGBoost_BAG_L1')]

# 데이터셋 및 모델 로드
train = pd.read_csv(f'./final_dataset/combined_data_HS_fin.csv')
X_train, X_test, y_train, y_test = train_test_split(train.drop(['ID', 'REAL_VAL','TYPE'], axis=1), train['REAL_VAL'], test_size=0.2, random_state=seed)

pred_dic = {}

X_test = X_test.iloc[:5,:]

for (problem_type_, model_name_) in loop_ls:
    if problem_type_ == 'regression':
        # 물성값 예측
        yCols=['HS','SG','TS','EB']#,'MNY','REHO_MIN','REHO_MAX','REHO_TS2','REHO_TC90','SCR']
        data_type_ = 'SFT_regr'
    else:
        # 불량여부 예측
        yCols=['HS_RESULT','SG_RESULT','TS_RESULT','EB_RESULT']#,'MNY_RESULT','REHO_RESULT','SCR_RESULT']
        data_type_ = 'SFT_clas'

    for yCol in yCols:
        # 모델 로드    
        predictor = TabularPredictor.load(path=f'{data_type_}/combined_{yCol}_models', verbosity = 0)

        # 테스트 세트에 대한 예측
        pred_dic[yCol] = predictor.predict(X_test, model=model_name_).tolist()
        display(f"'{yCol}': ['{data_type_}/combined_{yCol}_models', '{model_name_}']")
pred_dic

"'HS': ['SFT_regr/combined_HS_models', 'LightGBM_BAG_L2']"

"'SG': ['SFT_regr/combined_SG_models', 'LightGBM_BAG_L2']"

"'TS': ['SFT_regr/combined_TS_models', 'LightGBM_BAG_L2']"

"'EB': ['SFT_regr/combined_EB_models', 'LightGBM_BAG_L2']"

"'HS_RESULT': ['SFT_clas/combined_HS_RESULT_models', 'XGBoost_BAG_L1']"

"'SG_RESULT': ['SFT_clas/combined_SG_RESULT_models', 'XGBoost_BAG_L1']"

"'TS_RESULT': ['SFT_clas/combined_TS_RESULT_models', 'XGBoost_BAG_L1']"

"'EB_RESULT': ['SFT_clas/combined_EB_RESULT_models', 'XGBoost_BAG_L1']"

CPU times: user 22.3 s, sys: 2.63 s, total: 24.9 s
Wall time: 6.28 s


{'HS': [68.35274505615234,
  72.52928924560547,
  61.74452590942383,
  67.82764434814453,
  57.74105453491211],
 'SG': [1.1907382011413574,
  1.0847678184509277,
  1.3059877157211304,
  1.2382798194885254,
  1.0756925344467163],
 'TS': [151.11993408203125,
  183.24851989746094,
  112.12493896484375,
  181.4371337890625,
  182.76068115234375],
 'EB': [584.5527954101562,
  263.2105407714844,
  511.177490234375,
  441.7354431152344,
  460.2165832519531],
 'HS_RESULT': [1.0, 1.0, 1.0, 1.0, 1.0],
 'SG_RESULT': [1.0, 1.0, 1.0, 1.0, 1.0],
 'TS_RESULT': [1.0, 1.0, 1.0, 1.0, 1.0],
 'EB_RESULT': [1.0, 1.0, 1.0, 1.0, 1.0]}